<a href="https://colab.research.google.com/github/PadmarajBhat/Real-Time-Analytics-on-Hadoop/blob/master/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pyspark
!pip install -U pyarrow
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java

     |████████████████████████████████| 215.6MB 106kB/s 
     |████████████████████████████████| 204kB 40.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLlib").getOrCreate()
a = [(0,1),(0,2),(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]
b = spark.createDataFrame(a,["class","num"])
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|  1|
|    0|  2|
|    1|  1|
|    1|  2|
|    2|  1|
|    2|  2|
|    3|  1|
|    3|  2|
+-----+---+



In [3]:
b= b.withColumn("num", b['num']/b['num'])
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|1.0|
|    0|1.0|
|    1|1.0|
|    1|1.0|
|    2|1.0|
|    2|1.0|
|    3|1.0|
|    3|1.0|
+-----+---+



In [4]:
b=b.withColumn("new", b['class']+b['num'])
b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



In [5]:

b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



In [6]:
!echo $PATH

/usr/local/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [7]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2300.000
BogoMIPS:            4600.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd i

In [0]:
#!less /proc/cpuinfo

Nice Article on UDFs: https://www.slideshare.net/ueshin/apache-arrow-and-pandas-udf-on-apache-spark

* When is SCALAR needed?

In [9]:
b.select('num').show()

+---+
|num|
+---+
|1.0|
|1.0|
|1.0|
|1.0|
|1.0|
|1.0|
|1.0|
|1.0|
+---+



In [10]:
b.withColumn("num_2", b["num"]* 5).show()

+-----+---+---+-----+
|class|num|new|num_2|
+-----+---+---+-----+
|    0|1.0|1.0|  5.0|
|    0|1.0|1.0|  5.0|
|    1|1.0|2.0|  5.0|
|    1|1.0|2.0|  5.0|
|    2|1.0|3.0|  5.0|
|    2|1.0|3.0|  5.0|
|    3|1.0|4.0|  5.0|
|    3|1.0|4.0|  5.0|
+-----+---+---+-----+



In [11]:
b.withColumn("str(num)", str(b['num']))

AssertionError: ignored

Create UDF to convert number to string

In [0]:
from pyspark.sql.functions import col, pandas_udf, concat, lit
from pyspark.sql.types import LongType, StringType

# Declare the function and create the UDF
def num_to_string(a):
    b= a.apply(lambda x: str(x)+"_1")
    #print(b)
    return b

numToString = pandas_udf(num_to_string, returnType=StringType())

In [13]:
b.withColumn("new_num", numToString(col('num'))).show()

+-----+---+---+-------+
|class|num|new|new_num|
+-----+---+---+-------+
|    0|1.0|1.0|  1.0_1|
|    0|1.0|1.0|  1.0_1|
|    1|1.0|2.0|  1.0_1|
|    1|1.0|2.0|  1.0_1|
|    2|1.0|3.0|  1.0_1|
|    2|1.0|3.0|  1.0_1|
|    3|1.0|4.0|  1.0_1|
|    3|1.0|4.0|  1.0_1|
+-----+---+---+-------+



In [14]:
b.select(concat(b['num'].cast(StringType()),lit("_1"))  ).show()

+-------------------------------+
|concat(CAST(num AS STRING), _1)|
+-------------------------------+
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
+-------------------------------+



In [15]:
b.withColumn("numToString",concat(b['num'].cast(StringType()),lit("_1"))  ).show()

+-----+---+---+-----------+
|class|num|new|numToString|
+-----+---+---+-----------+
|    0|1.0|1.0|      1.0_1|
|    0|1.0|1.0|      1.0_1|
|    1|1.0|2.0|      1.0_1|
|    1|1.0|2.0|      1.0_1|
|    2|1.0|3.0|      1.0_1|
|    2|1.0|3.0|      1.0_1|
|    3|1.0|4.0|      1.0_1|
|    3|1.0|4.0|      1.0_1|
+-----+---+---+-----------+



Let us move on for grouped_map ; hope to see the use of scalar map in future.
For now, I am assuming that the time taken for the same is different and pandas function must be faster than sql functions

In [16]:
%%timeit
b.select( numToString(col('num'))).show()

+------------------+
|num_to_string(num)|
+------------------+
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
+------------------+

+------------------+
|num_to_string(num)|
+------------------+
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
+------------------+

+------------------+
|num_to_string(num)|
+------------------+
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
+------------------+

+------------------+
|num_to_string(num)|
+------------------+
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
|             1.0_1|
+---------

In [17]:
%%timeit
b.select(concat(b['num'].cast(StringType()),lit("_1"))  ).show()

+-------------------------------+
|concat(CAST(num AS STRING), _1)|
+-------------------------------+
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
+-------------------------------+

+-------------------------------+
|concat(CAST(num AS STRING), _1)|
+-------------------------------+
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
|                          1.0_1|
+-------------------------------+

+-------------------------------+
|concat(CAST(num AS STRING), _1)|
+-------------------------------+
|                          1.0_1|
|                          1.0_1|
|           

The above experiment do not conclude that either of the functions( pandas or sql) is faster. In order to conclude that we need not only a huge data set but also multiple runs to conclude the same.

Hence, I will continue to assume there may be a difference in performance when it comes to 2 functions.

**GROUPED_MAP**

In [18]:
b.groupBy(col('class')).apply(lambda x: x).show()

ValueError: ignored

### Note: Pandas DataFrame.assign not only creates new column but also creates a copy of it.

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf("class long, num double, new long", PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
  
  return pdf.assign(num=pdf.num*2)

@pandas_udf("class long, num double", PandasUDFType.GROUPED_MAP)
def mutltipl_3(pdf):
  return pdf.assign(num=pdf.num*3)

In [20]:
b.groupBy("num").apply(subtract_mean).show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|2.0|  1|
|    0|2.0|  1|
|    1|2.0|  2|
|    1|2.0|  2|
|    2|2.0|  3|
|    2|2.0|  3|
|    3|2.0|  4|
|    3|2.0|  4|
+-----+---+---+



In [21]:
b.groupBy("num").apply(mutltipl_3).show()

Py4JJavaError: ignored

In [22]:
b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



So it is not only works on the multiple columns returning same shape  but also works split-apply-combine functions.

Splits based on groupby clause and can cause overflow if the dataset for the id selected in groupby is skewed.

Again here these operation can be done parallelly by sql functions too but moving towards data science architecture like it in panda syntax rather to sql syntax.

Also it is important to note that we ***need***  give all the columns in the grouped map.

#### GROUPED_AGG

In [23]:
@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def mean_udf(v):
    return v.mean()
  
b.groupBy("class").agg(mean_udf('new')).show()

+-----+-------------+
|class|mean_udf(new)|
+-----+-------------+
|    0|          1.0|
|    1|          2.0|
|    3|          4.0|
|    2|          3.0|
+-----+-------------+



#### Last but not the least, the SQL queries on dataframe for SQL lovers.

In [0]:
b.createOrReplaceTempView("TempTable")

In [25]:
spark.sql("select * from TempTable").show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



In [26]:
spark.sql("select class, avg(new) from TempTable group by class").show()

+-----+--------+
|class|avg(new)|
+-----+--------+
|    0|     1.0|
|    1|     2.0|
|    3|     4.0|
|    2|     3.0|
+-----+--------+



### ETL
https://medium.com/@mrpowers/how-to-write-spark-etl-processes-df01b0c1bec9

In [0]:
from os.path import expanduser, join, abspath

from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()


In [28]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")

AnalysisException: ignored

#### Since we do not hve support, we cannot execute here. but is it any use?

https://spark.apache.org/docs/latest/sql-data-sources-hive-tables.html


df.write.partitionBy("key").format("hive").saveAsTable("hive_part_tbl")


#### Deep Learning at Spark
* https://github.com/yahoo/TensorFlowOnSpark/blob/master/examples/mnist/spark/mnist_spark.py

* https://github.com/tensorflow/ecosystem/tree/master/spark/spark-tensorflow-connector#usage-examples